#### Michael Hotaling <br> DSC540 <br> 2020-12-07

In [239]:
import pandas as pd
import numpy as np
import requests
import os
import time
import finviz as fz
import re
import requests
import unicodedata
from bs4 import BeautifulSoup
from re import search

### Scraping Data from SEC EDGAR
---
Every public company is required to submit quarterly and annual earnings reports to the Securities and Exchange Commission (SEC). The SEC provides this information free of charge on their website (https://www.sec.gov/edgar/search/) We are able to scrape this data to create a database of all of the quarterly and annual filings of every public company for future analysis.

Documentation on navigating the EDGAR can be found here: https://www.sec.gov/edgar/searchedgar/accessing-edgar-data.htm

We can first start by downloading the `master.idx` index file, which contains the Central Index Key (CIK) code for each company, the company name, the type of form that was filed, the date the form was filed, and the directory of where we can find the file.

In [378]:
url = 'https://www.sec.gov/Archives/edgar/full-index/master.idx'
r = requests.get(url, stream=True)
with open("master.txt", 'wb') as f:
    for chunk in r.iter_content(chunk_size=10240):
        f.write(chunk)

In [383]:
pd.read_table("master.txt", skiprows=11, sep="|", names = ["CIK", "Company Name", "Form Type","Date Filed","Filename"])

,CIK,Company Name,Form Type,Date Filed,Filename
0,1000045,NICHOLAS FINANCIAL INC,10-Q,2020-11-12,edgar/data/1000045/0001564590-20-053323.txt
1,1000045,NICHOLAS FINANCIAL INC,4,2020-11-04,edgar/data/1000045/0001720104-20-000006.txt
2,1000045,NICHOLAS FINANCIAL INC,4,2020-11-06,edgar/data/1000045/0001654216-20-000002.txt
3,1000045,NICHOLAS FINANCIAL INC,4,2020-11-06,edgar/data/1000045/0001720104-20-000007.txt
4,1000045,NICHOLAS FINANCIAL INC,4,2020-11-09,edgar/data/1000045/0001720104-20-000008.txt
...,...,...,...,...,...
165172,9984,BARNES GROUP INC,4,2020-11-13,edgar/data/9984/0000009984-20-000122.txt
165173,9984,BARNES GROUP INC,4,2020-11-20,edgar/data/9984/0000009984-20-000124.txt
165174,9984,BARNES GROUP INC,8-K,2020-10-13,edgar/data/9984/0000009984-20-000109.txt
165175,9984,BARNES GROUP INC,8-K,2020-10-23,edgar/data/9984/0000009984-20-000112.txt


In [398]:
pd.read_table("master.txt", skiprows=11, sep="|", names = ["CIK", "Company Name", "Form Type","Date Filed","Filename"])['Form Type'].value_counts()

4            55558
8-K          12761
424B2        12224
NPORT-P       8738
3             7707
             ...  
1-U/A            1
40FR12G          1
N-18F1           1
NT N-MFP2        1
N-23C3B          1
Name: Form Type, Length: 308, dtype: int64

The most up-to-date `master.idx` file contains every filing for this quarter, including quarterly filings (10-Q), annual filings for companies finishing their fiscal year this quarter (10-K), Form 4, which is a form required for majority shareholders to submit whenever they buy or sell stock, and several other types of forms. 

We will be most interested in 10-Q and 10-K forms, as well as Form 4 in the future, so for now to save memory, we can work with only the `xbrl.idx` file. To get all of the forms we want going back several years, we can iteratively go over each folder in the directory and write it to our disk

In [399]:
years = range(2000,2021)

quarters = ['QTR1', 'QTR2', 'QTR3', 'QTR4']

base_path = 'C:\\Users\\hotal\\Documents\\DSC540\\Finals'

current_dirs = os.listdir(path=base_path)

for yr in years:
    if str(yr) not in current_dirs:
        os.mkdir('/'.join([base_path, str(yr)]))
    current_files = os.listdir('/'.join([base_path, str(yr)]))
    
    for qtr in quarters:
        local_filename =  f'xbrl-index-{yr}-{qtr}.txt'
        local_file_path = '/'.join([base_path, str(yr), local_filename])
        if local_filename in current_files:
            continue
        url = f'https://www.sec.gov/Archives/edgar/full-index/{yr}/{qtr}/xbrl.idx'
        r = requests.get(url, stream=True)
        with open(local_file_path, 'wb') as f:
            for chunk in r.iter_content(chunk_size=10240):
                f.write(chunk)
        time.sleep(1)

Since ticker information isn't available on this dataset, we can pull the ticker index file from the SEC and cross correlate it with the CIK number associated with that ticker to get the company information

In [400]:
def ticker_downloader():
    tickers = pd.read_table("https://www.sec.gov/include/ticker.txt", names = ['Ticker', "CIK"])
    tickers.to_csv("ticker.csv")
    
ticker_downloader()

In [401]:
def ticker_to_cik(ticker):
    """Returns the CIK value for a given ticker"""
    tickers = pd.read_csv("ticker.csv")
    ticker = ticker.replace(".","-")
    return int(tickers[tickers['Ticker'] == ticker.lower()]["CIK"])

In [402]:
ticker_to_cik("AAPL")

320193

We can now consolidate all of our `xbrl.idx` files into one dataframe 

In [424]:
super_frame = pd.DataFrame(columns = ["CIK", "Company Name", "Form Type","Date Filed","Filename"])

for i in range(2000,2021):
    for j in range(1,5):
        df = pd.read_table("{}/xbrl-index-{}-QTR{}.txt".format(i,i,j),sep="|", skiprows=10, 
                           names = ["CIK", "Company Name", "Form Type","Date Filed","Filename"])
        super_frame = super_frame.append(df, ignore_index=True)
super_frame

,CIK,Company Name,Form Type,Date Filed,Filename
0,1107194,CENTILLIUM COMMUNICATIONS INC,SC TO-I/A,2004-12-13,edgar/data/1107194/0000891618-04-001377.txt
1,1000048,TECHNOLOGY FLAVORS & FRAGRANCES INC,SC TO-T/A,2005-05-26,edgar/data/1000048/0001193125-05-115670.txt
2,1067491,INFOSYS TECHNOLOGIES LTD,6-K,2005-05-03,edgar/data/1067491/0001193125-05-092677.txt
3,1080224,EDGAR ONLINE INC,8-K,2005-04-25,edgar/data/1080224/0001275287-05-001434.txt
4,1271941,SILVERBOIM HOLDINGS LTD,SC TO-T/A,2005-06-30,edgar/data/1271941/0000950136-05-003917.txt
...,...,...,...,...,...
425824,99780,TRINITY INDUSTRIES INC,8-K,2020-12-04,edgar/data/99780/0000099780-20-000144.txt
425825,9984,BARNES GROUP INC,10-Q,2020-10-26,edgar/data/9984/0000009984-20-000115.txt
425826,9984,BARNES GROUP INC,8-K,2020-10-13,edgar/data/9984/0000009984-20-000109.txt
425827,9984,BARNES GROUP INC,8-K,2020-10-23,edgar/data/9984/0000009984-20-000112.txt


---

We can filter our dataframe to pull all of the data to access our 10-Q and 10-K forms. We can use Facebook (FB) as an example for this exercise.

In [425]:
desired_ticker = "FB"

super_frame[((super_frame['Form Type'] == "10-K")|(super_frame['Form Type'] == "10-Q")) & 
            (super_frame['CIK'] == ticker_to_cik(desired_ticker))]

,CIK,Company Name,Form Type,Date Filed,Filename
52147,1326801,Facebook Inc,10-Q,2012-07-31,edgar/data/1326801/0001193125-12-325997.txt
61430,1326801,Facebook Inc,10-Q,2012-10-24,edgar/data/1326801/0001326801-12-000006.txt
70656,1326801,Facebook Inc,10-K,2013-02-01,edgar/data/1326801/0001326801-13-000003.txt
79823,1326801,Facebook Inc,10-Q,2013-05-02,edgar/data/1326801/0001326801-13-000011.txt
90573,1326801,Facebook Inc,10-Q,2013-07-25,edgar/data/1326801/0001326801-13-000019.txt
100158,1326801,Facebook Inc,10-Q,2013-11-01,edgar/data/1326801/0001326801-13-000031.txt
109488,1326801,Facebook Inc,10-K,2014-01-31,edgar/data/1326801/0001326801-14-000007.txt
118655,1326801,Facebook Inc,10-Q,2014-04-25,edgar/data/1326801/0001326801-14-000023.txt
128513,1326801,Facebook Inc,10-Q,2014-07-24,edgar/data/1326801/0001326801-14-000032.txt
137729,1326801,Facebook Inc,10-Q,2014-10-30,edgar/data/1326801/0001326801-14-000068.txt


The information given doesn't give us a CSV file of where the information we want is, it returns to a text file containing all of the HTML code within that filing, which can be difficult to process. 

The information we want is also available in JSON format, so we can use the same directory provided, but just change the formatting a little.

In [426]:
normal_url = "https://www.sec.gov/Archives/" + super_frame[(super_frame['Form Type'] == "10-K") & (super_frame['CIK'] == ticker_to_cik(desired_ticker))]['Filename'].tail(1).values[0]
print(normal_url)

https://www.sec.gov/Archives/edgar/data/1326801/0001326801-20-000013.txt


In [427]:
normal_url = normal_url.replace("-","").replace(".txt",'/index.json')
print(normal_url)

https://www.sec.gov/Archives/edgar/data/1326801/000132680120000013/index.json


In [428]:
content = requests.get(normal_url).json()

In [429]:
pd.DataFrame(content)

,directory
item,"[{'last-modified': '2020-01-29 21:12:14', 'nam..."
name,/Archives/edgar/data/1326801/000132680120000013
parent-dir,/Archives/edgar/data/1326801


We need to find the `FilingSummary.xml`, which contains the URLs of each table inside the report. We can then parse our the tables that are important to us.

In [430]:
for file in content['directory']['item']:
    if file['name'] == 'FilingSummary.xml':
        xml_summary = "https://www.sec.gov" + content['directory']['name'] + "/" + file['name']
        print(xml_summary)

https://www.sec.gov/Archives/edgar/data/1326801/000132680120000013/FilingSummary.xml


In [431]:
base_url = xml_summary.replace('FilingSummary.xml',"")
content = requests.get(xml_summary).content
soup = BeautifulSoup(content, 'lxml')

In [432]:
reports = soup.find('myreports')

In [433]:
master_reports = []

for report in reports.find_all('report')[:-1]:
    report_dict = {}
    report_dict['shortname'] = report.shortname.text
    report_dict['longname'] = report.longname.text
    report_dict['position'] = report.position.text
    report_dict['menucategory'] = report.menucategory.text
    report_dict['filename'] = base_url + report.htmlfilename.text
    
    master_reports.append(report_dict)

In [438]:
testdf = pd.DataFrame(master_reports)
testdf

,shortname,longname,position,menucategory,filename
0,Cover page,0001000 - Document - Cover page,1,Cover,https://www.sec.gov/Archives/edgar/data/132680...
1,CONSOLIDATED BALANCE SHEETS,1001000 - Statement - CONSOLIDATED BALANCE SHEETS,2,Statements,https://www.sec.gov/Archives/edgar/data/132680...
2,CONSOLIDATED BALANCE SHEETS (Parenthetical),1001501 - Statement - CONSOLIDATED BALANCE SHE...,3,Statements,https://www.sec.gov/Archives/edgar/data/132680...
3,CONSOLIDATED STATEMENTS OF INCOME,1002000 - Statement - CONSOLIDATED STATEMENTS ...,4,Statements,https://www.sec.gov/Archives/edgar/data/132680...
4,CONSOLIDATED STATEMENTS OF COMPREHENSIVE INCOME,1003000 - Statement - CONSOLIDATED STATEMENTS ...,5,Statements,https://www.sec.gov/Archives/edgar/data/132680...
...,...,...,...,...,...
65,Income Taxes - Schedule of Deferred Tax Asse...,2414405 - Disclosure - Income Taxes - Schedu...,66,Details,https://www.sec.gov/Archives/edgar/data/132680...
66,Income Taxes - Narrative (Details),2414406 - Disclosure - Income Taxes - Narrati...,67,Details,https://www.sec.gov/Archives/edgar/data/132680...
67,Income Taxes - Schedule of Unrecognized Tax B...,2414407 - Disclosure - Income Taxes - Schedul...,68,Details,https://www.sec.gov/Archives/edgar/data/132680...
68,Geographical Information - Schedule of Proper...,2415402 - Disclosure - Geographical Informatio...,69,Details,https://www.sec.gov/Archives/edgar/data/132680...


For now, we are only really interested in extracting information from the Statements tables. We can parse the four most important tables, the Balance Sheet, the Income Statement, the Cash Flows, and the Stockholder Equity tables.

The names of these tables are not universal, so we will need to be careful when trying to extract information from other companies.

In [443]:
statements_url = []

for report_dict in master_reports:
    """Everything is named differently so I'll need to do some fancy regex work to get this to work"""
    
    item1 = "Consolidated Balance Sheets".upper()
    item2 = "CONSOLIDATED STATEMENTs OF INCOME".upper()
    item3 = "Consolidated Statements of Cash Flows".upper()
    item4 = "Consolidated Statements of Stockholders' Equity".upper()
    
    report_list = [item1,item2,item3,item4]
    
    if report_dict['shortname'].upper() in report_list:
        print(report_dict['shortname'])
        print(report_dict['filename'])
        statements_url.append(report_dict['filename'])

CONSOLIDATED BALANCE SHEETS
https://www.sec.gov/Archives/edgar/data/1326801/000132680120000013/R2.htm
CONSOLIDATED STATEMENTS OF INCOME
https://www.sec.gov/Archives/edgar/data/1326801/000132680120000013/R4.htm
CONSOLIDATED STATEMENTS OF STOCKHOLDERS' EQUITY
https://www.sec.gov/Archives/edgar/data/1326801/000132680120000013/R6.htm
CONSOLIDATED STATEMENTS OF CASH FLOWS
https://www.sec.gov/Archives/edgar/data/1326801/000132680120000013/R7.htm


The tables above have a strange formatting due to generally accepted accounting practices (GAAP). We can use BeautifulSoup to reorganize the table into an interpretable dataframe.

In [483]:
statements_data = []

for statement in statements_url:
    statement_data = {}
    statement_data['header'] = []
    statement_data['section'] = []
    statement_data['data'] = []
    content = requests.get(statement).content
    report_soup = BeautifulSoup(content,"html")
    for index, row in enumerate(report_soup.table.find_all('tr')):
        cols = row.find_all('td')
        if (len(row.find_all('th'))) == 0 and (len(row.find_all('strong')) == 0):
            reg_row = [ele.text.strip() for ele in cols]
            statement_data['data'].append(reg_row)
        elif (len(row.find_all('th')) == 0 and len(row.find_all('strong')) != 0):
            sec_row = cols[0].text.strip()
            statement_data['section'].append(sec_row)
        elif (len(row.find_all('th'))!= 0):
            hed_row = [ele.text.strip() for ele in row.find_all('th')]
            statement_data['header'].append(hed_row)
    statements_data.append(statement_data)
    
pd.DataFrame(statements_data)

,header,section,data
0,[[CONSOLIDATED BALANCE SHEETS - USD ($) $ in M...,"[Current assets:, Current liabilities:, Stockh...","[[Cash and cash equivalents, $ 19,079, $ 10,01..."
1,[[CONSOLIDATED STATEMENTS OF INCOME - USD ($) ...,"[Costs and expenses:, Earnings per share attri...","[[Revenue, $ 70,697, $ 55,838, $ 40,653], [Cos..."
2,[[CONSOLIDATED STATEMENTS OF STOCKHOLDERS' EQU...,[Increase (Decrease) in Stockholders' Equity [...,"[[Common stock, shares outstanding beginning (..."
3,[[CONSOLIDATED STATEMENTS OF CASH FLOWS - USD ...,"[Cash flows from operating activities, Adjustm...","[[Net income, $ 18,485, $ 22,112, $ 15,934], [..."


In [485]:
income_header = statements_data[1]['header'][1]
income_data = statements_data[1]['data']

income_df = pd.DataFrame(income_data)

income_df.index = income_df[0]
income_df.index.name = 'Category'
income_df = income_df.drop(0,axis=1)
income_df = income_df.replace('[\$,)]',"", regex=True)
income_df = income_df.replace('[(]',"-", regex = True)
income_df = income_df.replace('', 'NaN', regex = True)
income_df = income_df.astype(float)
income_df.columns = income_header

income_df

,"Dec. 31, 2019","Dec. 31, 2018","Dec. 31, 2017"
Category,,,
Revenue,70697.00,55838.00,40653.00
Cost of revenue,12770.00,9355.00,5454.00
Research and development,13600.00,10273.00,7754.00
Marketing and sales,9876.00,7846.00,4725.00
General and administrative,10465.00,3451.00,2517.00
Total costs and expenses,46711.00,30925.00,20450.00
Income from operations,23986.00,24913.00,20203.00
"Interest and other income, net",826.00,448.00,391.00
Income before provision for income taxes,24812.00,25361.00,20594.00


In [486]:
income_header = statements_data[3]['header'][1]
income_data = statements_data[3]['data']

income_df = pd.DataFrame(income_data)

income_df.index = income_df[0]
income_df.index.name = 'Category'
income_df = income_df.drop(0,axis=1)
income_df = income_df.replace('[\$,)]',"", regex=True)
income_df = income_df.replace('[(]',"-", regex = True)
income_df = income_df.replace('', 'NaN', regex = True)
income_df = income_df.astype(float)
income_df.columns = income_header

income_df

,"Dec. 31, 2019","Dec. 31, 2018","Dec. 31, 2017"
Category,,,
Net income,18485.0,22112.0,15934.0
Depreciation and amortization,5741.0,4315.0,3025.0
Share-based compensation,4836.0,4152.0,3723.0
Deferred income taxes,-37.0,286.0,-377.0
Other,39.0,-64.0,24.0
Accounts receivable,-1961.0,-1892.0,-1609.0
Prepaid expenses and other current assets,47.0,-690.0,-192.0
Other assets,41.0,-159.0,154.0
Accounts payable,113.0,221.0,43.0


In [500]:
income_header = statements_data[0]['header'][0][1::]
income_data = statements_data[0]['data']

income_df = pd.DataFrame(income_data)

income_df.index = income_df[0]
income_df.index.name = 'Category'
income_df = income_df.drop(0,axis=1)
income_df = income_df.replace('[\$,)]',"", regex=True)
income_df = income_df.replace('[(]',"-", regex = True)
income_df = income_df.replace('', 'NaN', regex = True)
income_df = income_df.astype(float)
income_df.columns = income_header
income_df

,"Dec. 31, 2019","Dec. 31, 2018"
Category,,
Cash and cash equivalents,19079.0,10019.0
Marketable securities,35776.0,31095.0
"Accounts receivable, net of allowances of $206 and $229 as of December 31, 2019 and December 31, 2018, respectively",9518.0,7587.0
Prepaid expenses and other current assets,1852.0,1779.0
Total current assets,66225.0,50480.0
"Property and equipment, net",35323.0,NaN
"Property and equipment, net",NaN,24683.0
"Operating lease right-of-use assets, net",9460.0,NaN
"Intangible assets, net",894.0,1294.0


In [501]:
income_header = statements_data[2]['header'][0][1::]
income_data = statements_data[2]['data']

income_df = pd.DataFrame(income_data)

income_df.index = income_df[0]
income_df.index.name = 'Category'
income_df = income_df.drop(0,axis=1)
income_df = income_df.replace('[\$,)]',"", regex=True)
income_df = income_df.replace('[(]',"-", regex = True)
income_df = income_df.replace('', 'NaN', regex = True)
income_df = income_df.astype(float)
income_df.columns = income_header
income_df

,Total,Class A and Class B Common Stock,Additional Paid-In Capital,Accumulated Other Comprehensive Loss,Retained Earnings
Category,,,,,
"Common stock, shares outstanding beginning (in shares) at Dec. 31, 2016",NaN,2892.0,NaN,NaN,NaN
"Total stockholders' equity, beginning at Dec. 31, 2016",59194.0,0.0,38227.0,-703.0,21670.0
"Issuance of common stock related to acquisitions, shares",NaN,2.0,NaN,NaN,NaN
"Issuance of common stock related to acquisitions, value",323.0,NaN,323.0,NaN,NaN
"Issuance of common stock for cash upon exercise of stock options, shares",NaN,3.0,NaN,NaN,NaN
"Issuance of common stock for cash upon exercise of stock options, value",13.0,NaN,13.0,NaN,NaN
"Issuance of common stock for settlement of RSUs, shares",NaN,43.0,NaN,NaN,NaN
Issuance of common stock for settlement of RSUs. value,0.0,NaN,NaN,NaN,NaN
"Shares withheld related to net share settlement, shares",NaN,-21.0,NaN,NaN,NaN


Now that we've extracted the information once, we should be able to do it iteratively over the past several years and append that data to our dataframe for future analysis. 